In [1]:
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install folium



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# 1. 필요한 라이브러리 불러오기

import collections
import datetime
import glob
import math
import os
import pathlib

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import folium


pd.options.display.float_format = '{:,}'.format

plt.rcParams['font.family'] ='D2Coding'
plt.rcParams['axes.unicode_minus'] =False

DAY_OF_WEEK = ['월', '화', '수', '목', '금', '토', '일']

BASE_DIR = pathlib.Path('../').resolve()
print(os.listdir(BASE_DIR / '실습' / 'rawfiles'))

palette = sns.color_palette('husl', n_colors=7)
palette

['CARD_SUBWAY_MONTH_201903.csv', 'CARD_SUBWAY_MONTH_201902.csv', 'CARD_SUBWAY_MONTH_201901.csv', 'CARD_SUBWAY_MONTH_201905.csv', 'CARD_SUBWAY_MONTH_201904.csv', 'CARD_SUBWAY_MONTH_201906.csv', '지하철노선위경도정보2.csv']


[(0.9677975592919913, 0.44127456009157356, 0.5358103155058701),
 (0.7757319041862729, 0.5784925270759935, 0.19475566538551875),
 (0.5105309046900421, 0.6614299289084904, 0.1930849118538962),
 (0.20433460114757862, 0.6863857739476534, 0.5407103379425205),
 (0.21662978923073606, 0.6676586160122123, 0.7318695594345369),
 (0.5049017849530067, 0.5909119231215284, 0.9584657252128558),
 (0.9587050080494409, 0.3662259565791742, 0.9231469575614251)]

In [3]:
# 3. 데이터를 병합하여 하나의 데이터프레임 형성

df_subway_raw = pd.DataFrame()

for filename in glob.glob(str(BASE_DIR / '실습' / 'rawfiles' / 'CARD_SUBWAY_MONTH_*.csv')):
    tmp = pd.read_csv(filename)
    df_subway_raw = pd.concat([df_subway_raw, tmp])

df_subway_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107028 entries, 0 to 17723
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   사용일자    107028 non-null  int64  
 1   노선명     107028 non-null  object 
 2   역명      107028 non-null  object 
 3   승차총승객수  107028 non-null  float64
 4   하차총승객수  107028 non-null  float64
 5   등록일자    107028 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 5.7+ MB


In [4]:
# dt.dayofweek로 표현된 요일의 번호가 0~6까지 각각 월~일에 대응되는지 확인해보자.

df = df_subway_raw.copy()

df['사용일자'] = pd.to_datetime(df['사용일자'], format='%Y%m%d')
df['요일'] = df['사용일자'].dt.dayofweek

# 달력을 통해, 2019년 5월 5일 어린이날은 '일요일'인 것을 알고있는 상황.
# 2019년도 어린이 날에 해당하는 데이터만 필터링한다.
df = df[df['사용일자'] == '2019-05-05']

# 어린이 날(일요일)의 dayofweek(숫자)를 확인한다.
df.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일
2370,2019-05-05,4호선,서울역,"45,856.0","42,422.0",20190508,6
2371,2019-05-05,2호선,시청,"15,457.0","15,413.0",20190508,6
2372,2019-05-05,1호선,종각,"31,735.0","27,154.0",20190508,6
2373,2019-05-05,3호선,종로3가,"26,047.0","24,846.0",20190508,6
2374,2019-05-05,1호선,종로5가,"16,105.0","16,292.0",20190508,6


In [5]:
df_subway_raw['사용일자'] = pd.to_datetime(df_subway_raw['사용일자'], format='%Y%m%d')
df_subway_raw['주차'] = df_subway_raw['사용일자'].dt.dayofyear // 7 + 1
df_subway_raw['요일'] = df_subway_raw['사용일자'].dt.dayofweek.map({ idx: val for idx, val in enumerate(DAY_OF_WEEK) })
df_subway_raw['연'] = df_subway_raw['사용일자'].dt.year
df_subway_raw['월'] = df_subway_raw['사용일자'].dt.month
df_subway_raw['일'] = df_subway_raw['사용일자'].dt.day
df_subway_raw['승하차총승객수'] = (df_subway_raw['승차총승객수'] + df_subway_raw['하차총승객수']).astype(int)

df_subway_raw.sample(10)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,주차,요일,연,월,일,승하차총승객수
15045,2019-04-26,경춘선,평내호평,"6,953.0","6,999.0",20190429,17,금,2019,4,26,13952
1042,2019-02-02,경의선,지평,25.0,37.0,20190205,5,토,2019,2,2,62
9668,2019-03-17,2호선,충정로,"2,007.0","1,729.0",20190320,11,일,2019,3,17,3736
4815,2019-01-09,3호선,대청,"11,097.0","10,047.0",20190112,2,수,2019,1,9,21144
12575,2019-06-22,2호선,합정,"34,793.0","39,327.0",20190625,25,토,2019,6,22,74120
18204,2019-01-31,7호선,마들,"13,822.0","12,835.0",20190203,5,목,2019,1,31,26657
7276,2019-03-13,5호선,상일동,"15,737.0","13,211.0",20190316,11,수,2019,3,13,28948
6046,2019-01-11,3호선,매봉,"14,024.0","13,433.0",20190114,2,금,2019,1,11,27457
5138,2019-01-09,5호선,굽은다리,"10,537.0","10,056.0",20190112,2,수,2019,1,9,20593
17845,2019-01-31,4호선,길음,"24,393.0","23,208.0",20190203,5,목,2019,1,31,47601


In [6]:
holidays = [
    datetime.date(2019, 1, 1),  # 신정
    datetime.date(2019, 2, 4),  # 설날(연휴)
    datetime.date(2019, 2, 5),  # 설날
    datetime.date(2019, 2, 6),  # 설날(연휴)
    datetime.date(2019, 3, 1),  # 삼일절
    datetime.date(2019, 5, 5),  # 어린이날
    datetime.date(2019, 5, 6),  # (어린이날)대체휴일
    datetime.date(2019, 5, 12),  # 부처님오신날
    datetime.date(2019, 6, 6),  # 현충일
    datetime.date(2019, 8, 15),  # 광복절
    datetime.date(2019, 9, 12),  # 추석(연휴)
    datetime.date(2019, 9, 13),  # 추석
    datetime.date(2019, 9, 14),  # 추석(연휴)
    datetime.date(2019, 10, 3),  # 개천절
    datetime.date(2019, 10, 9),  # 한글날
    datetime.date(2019, 12, 25),  # 크리스마스
]

df_subway_raw['공휴일'] = df_subway_raw['일'].isin(holidays)

df_subway_raw.sample()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,주차,요일,연,월,일,승하차총승객수,공휴일
13715,2019-06-24,3호선,매봉,"14,324.0","13,773.0",20190627,26,월,2019,6,24,28097,False


In [7]:
df_location_raw = pd.read_csv(str(BASE_DIR / '실습' / 'rawfiles' / '지하철노선위경도정보2.csv'))
df_location_raw['역명'] = df_location_raw['역이름']
del df_location_raw['역이름']
df_location_raw.sample()

,역지역,위도,경도,호선,역명
279,수도권,37.47540683,126.6326025,1호선,동인천


In [8]:
df_subway_raw = df_subway_raw.merge(df_location_raw, on='역명', how='left')
df_subway_raw.sample(5)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,주차,요일,연,월,일,승하차총승객수,공휴일,역지역,위도,경도,호선
127790,2019-04-30,2호선,교대,"18,984.0","12,756.0",20190503,18,화,2019,4,30,31740,False,대구,35.85063736,128.5905762,1호선
22677,2019-03-27,1호선,세류,"4,209.0","4,081.0",20190330,13,수,2019,3,27,8290,False,수도권,37.24387252,127.0137952,1호선
136313,2019-06-10,3호선,고속터미널,"20,495.0","17,967.0",20190613,24,월,2019,6,10,38462,False,수도권,37.50453984,127.0051346,9호선
128533,2019-06-01,경의선,디지털미디어시티,"9,222.0","9,557.0",20190604,22,토,2019,6,1,18779,False,수도권,37.57600202,126.9012631,경의선
11655,2019-03-14,6호선,이태원,"15,062.0","16,746.0",20190317,11,목,2019,3,14,31808,False,수도권,37.53450718,126.9943203,6호선


In [9]:
df_subway_raw['노선명'] = df_subway_raw['호선']
del df_subway_raw['호선']
df_subway_raw.sample(5)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,주차,요일,연,월,일,승하차총승객수,공휴일,역지역,위도,경도
33428,2019-02-09,분당선,신갈,"3,722.0","3,529.0",20190212,6,토,2019,2,9,7251,False,수도권,37.28613395,127.1112802
54689,2019-01-06,경춘선,사릉,"1,847.0","1,844.0",20190109,1,일,2019,1,6,3691,False,수도권,37.65110718,127.1769126
105097,2019-04-04,3호선,교대,"44,388.0","50,427.0",20190407,14,목,2019,4,4,94815,False,수도권,37.49273596,127.0138634
40415,2019-02-17,6호선,증산,"6,894.0","6,698.0",20190220,7,일,2019,2,17,13592,False,수도권,37.58373913,126.9094724
60076,2019-01-12,6호선,공덕,"1,889.0","2,843.0",20190115,2,토,2019,1,12,4732,False,수도권,37.54453447,126.9514331


In [10]:
df_subway_raw.to_csv(str(BASE_DIR / 'data' / '지하철_데이터병합_2019.csv'), index=False)